In [9]:
import os
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
#from langgraph import ToolExecutor, ToolNode
from pydantic import BaseModel, Field

# 1. Define the Structured Output Model for the Summary
class ServerSummary(BaseModel):
    """
    Structured summary of server-related actions and network configurations extracted from text.
    """
    state_server_names: List[str] = Field(
        description="List of server names mentioned in the text."
    )
    state_server_os: List[str] = Field(
        description="List of server operating system mentioned in the text."
    )
    state_action: str = Field(
        description="Overall state action to be performed on servers (e.g., 'deploy', 'shutdown', 'configure')."
    )
    state_network_action: List[str] = Field(
        description="List of network actions mentioned (e.g., 'open port', 'block traffic', 'configure firewall')."
    )
    state_network_port: str = Field(
        description="Specific network port mentioned for network requests, if any. If multiple, pick the most prominent or state 'multiple'."
    )

# 2. Define the LangGraph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        file_path: The path to the input text file.
        file_content: The extracted content from the file.
        summary: The structured summary generated from the file content.
        error: Any error message encountered during processing.
    """
    file_path: str
    file_content: Annotated[str, "The content read from the file"]
    summary: Annotated[ServerSummary, "The structured summarized content"] # Updated type
    error: Annotated[str, "Any error message encountered"]


# 3. Define the Tools
@tool
def read_text_file(file_path: str) -> str:
    """
    Reads the content of a text file from the given file path.

    Args:
        file_path: The path to the text file.

    Returns:
        The content of the file as a string.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        return f"Error: File not found at {file_path}"
    except Exception as e:
        return f"Error reading file: {e}"

@tool
def summarize_text_structured(text: str) -> ServerSummary:
    """
    Summarizes the given input text into a structured ServerSummary object using an LLM.

    Args:
        text: The text to be summarized.

    Returns:
        A ServerSummary object containing extracted server and network details.
    """
    llm = ChatOpenAI(model="gpt-4o", temperature=0.1) # Lower temperature for more consistent output
    
    # Use with_structured_output to force the LLM to return a Pydantic object
    structured_llm = llm.with_structured_output(ServerSummary)

    prompt = f"""
    Analyze the following text to extract server-related information and network details.
    Populate the fields of the ServerSummary model based on the content.
    If information for a field is not explicitly available, return an empty list for lists
    or an empty string for strings, do NOT hallucinate.

    Text to analyze:
    {text}
    """
    try:
        response: ServerSummary = structured_llm.invoke(prompt)
        return response
    except Exception as e:
        # Return a default empty ServerSummary in case of LLM parsing error
        print(f"Error during structured summarization: {e}")
        return ServerSummary(
            state_server_names=[],
            state_action="",
            state_network_action=[],
            state_network_port=""
        )

# Register the tool (Note: using the new structured tool)
#tools = [read_text_file, summarize_text_structured]
#tool_executor = ToolExecutor(tools)

# 4. Define the Nodes

def extract_file_content_node(state: GraphState) -> GraphState:
    """
    LangGraph node to extract content from the file.
    """
    file_path = state["file_path"]
    print(f"Attempting to read file: {file_path}")
    content = read_text_file.invoke({"file_path": file_path})

    if content.startswith("Error:"):
        return {"error": content, "file_content": ""}
    else:
        return {"file_content": content, "error": ""}

def summarize_content_node(state: GraphState) -> GraphState:
    """
    LangGraph node to summarize the extracted file content into a structured format.
    """
    file_content = state["file_content"]
    if not file_content:
        print("No file content to summarize, skipping summarization.")
        # Return an empty ServerSummary if no content
        return {"summary": ServerSummary(state_server_names=[], state_action="", state_network_action=[], state_network_port=""), "error": "No content to summarize"}

    print("Summarizing file content into structured output...")
    # Call the structured summarization tool
    structured_summary = summarize_text_structured.invoke({"text": file_content})

    # Check if the returned object is indeed a ServerSummary (it should be due to with_structured_output)
    if isinstance(structured_summary, ServerSummary):
        return {"summary": structured_summary, "error": ""}
    else:
        # This case should ideally not happen if with_structured_output works as expected
        return {"error": "Failed to generate structured summary.", 
                "summary": ServerSummary(state_server_names=[], state_action="", state_network_action=[], state_network_port="")}

def decide_flow_node(state: GraphState) -> str:
    """
    LangGraph node to decide the next step based on the presence of errors.
    """
    if state.get("error"):
        print(f"Error detected: {state['error']}. Ending graph.")
        return "end" # Go to END if there's an error
    else:
        print("No errors, proceeding to summarize content.")
        return "summarize" # Proceed to summarization

# 5. Build the LangGraph Graph

workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("extract_content", extract_file_content_node)
workflow.add_node("summarize", summarize_content_node)

# Set entry point
workflow.set_entry_point("extract_content")

# Add edges
# After extraction, decide whether to summarize or end due to error
workflow.add_conditional_edges(
    "extract_content",
    decide_flow_node,
    {
        "summarize": "summarize",
        "end": END
    }
)

# After summarization, the process ends
workflow.add_edge("summarize", END)

# Compile the graph
app = workflow.compile()



In [12]:
# 6. Example Usage

# Create a more relevant dummy text file for testing structured output
dummy_file_path = "server_log_example.txt"

with open(dummy_file_path, "w", encoding="utf-8") as f:
    f.write("""
[2025-06-14 10:00:00] Initiating deployment to production servers: web-server-01, api-gateway-02.
[2025-06-14 10:05:00] Applying firewall rules. Opening port 80 for HTTP traffic.
[2025-06-14 10:10:00] Configuration complete for both servers. Monitoring status.
[2025-06-14 10:15:00] Traffic redirection successful. Testing connectivity on port 443.
[2025-06-14 11:00:00] Scheduled maintenance: Shutting down db-server-03 for disk upgrade.
""")

# Run the graph
print(f"\n--- Running Graph for '{dummy_file_path}' ---")
initial_state = {"file_path": "input.txt", "file_content": "", "summary": ServerSummary(state_server_names=[], state_server_os=[],state_action="", state_network_action=[], state_network_port=""), "error": ""}
final_state = app.invoke(initial_state)

if final_state.get("error"):
    print("\n--- Processing Failed ---")
    print(f"Error: {final_state['error']}")
else:
    print("\n--- Processing Complete ---")
    print("\nOriginal File Content (partial):")
    print(final_state['file_content'][:300] + "...") # Print first 300 chars
    
    print("\nStructured Summary:")
    summary_obj = final_state['summary']
    if isinstance(summary_obj, ServerSummary):
        print(f"  Server Names: {summary_obj.state_server_names}")
        print(f"  Server Operating Sytems: {summary_obj.state_server_os}")
        print(f"  Overall Action: {summary_obj.state_action}")
        print(f"  Network Actions: {summary_obj.state_network_action}")
        print(f"  Network Port: {summary_obj.state_network_port}")
    else:
        print("  Error: Summary is not in the expected structured format.")
        print(f"  Raw Summary: {summary_obj}")


# Test with a non-existent file
# non_existent_file_path = "non_existent_file.txt"
# print(f"\n--- Running Graph for '{non_existent_file_path}' (expecting error) ---")
# initial_state_error = {"file_path": non_existent_file_path, "file_content": "", "summary": ServerSummary(state_server_names=[], state_action="", state_network_action=[], state_network_port=""), "error": ""}
# final_state_error = app.invoke(initial_state_error)

# if final_state_error.get("error"):
#     print("\n--- Processing Failed (as expected) ---")
#     print(f"Error: {final_state_error['error']}")
# else:
#     print("\n--- Unexpected Success (should have failed) ---")
#     print(f"Summary: {final_state_error['summary']}")

# Clean up the dummy file
#os.remove(dummy_file_path)


--- Running Graph for 'server_log_example.txt' ---
Attempting to read file: input.txt
No errors, proceeding to summarize content.
Summarizing file content into structured output...

--- Processing Complete ---

Original File Content (partial):
build 2 new windows servers sin0m4app2069xd and sin0m4app2070xd and open commnication on port 1858
...

Structured Summary:
  Server Names: ['sin0m4app2069xd', 'sin0m4app2070xd']
  Server Operating Sytems: ['Windows']
  Overall Action: build
  Network Actions: ['open communication']
  Network Port: 1858
